In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Iniciar random
np.random.seed(6)

# Variables globales con información para acelerar procesamiento
g_filas = g_cols   = 0    # Serán números (1.000.000 y 11 en este caso)
g_pesos = g_dpesos = None # Serán arrays de 11 elementos
g_sesgo = g_dsesgo = 0    # Serán números (coma flotante)

ITER = 10
APRN = 0.5
LMBD = 0.000002

# Funciones

In [2]:
# linea es un string de la forma:
# "n0, n1, n2, ..., n10, r" donde nX es una propiedad del tráfico y r es
# su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    aux   = [float(i) for i in linea.split(',')]
    datos = np.array(aux[:-1])
    resul = aux[-1]
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn. La lectura con Spark
# ya es mucho más rápida que leyendo con numpy.loadtxt()
# Cada registro del RDD es una tupla (X, y)
#  X: np.array de los datos
#  y: clasificación (1 = botnet, 0 = no botnet)
def readFile(fn):
    global g_filas, g_cols
    sc      = SparkContext("local[*]", "BotnetParalelaPruebas")
    todo    = sc.textFile(fn)
    ret     = todo.map(formato_inicial)
    g_filas = todo.count()
    g_cols  = len(ret.take(1)[0][0])
    return ret

In [3]:
def norm_tupla(tuplaXy):
    media = np.mean(tuplaXy[0])
    stdev = np.std(tuplaXy[0])
    tnorm = (tuplaXy[0] - media)/stdev
    return (tnorm, tuplaXy[1])

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
# X es un array con 11 flotantes
# y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    return rdd.map(norm_tupla)

In [4]:
# datos: RDD de 1M de tuplas (X, y, ŷ)
# np.sum(res*np.log(pred_res + epsilon) + (1-res)*np.log(1-pred_res + epsilon))
def presio(datos, filas):
    epsilon = 1e-5
    return epsilon

# datos: RDD de 1M de tuplas (X, y, ŷ)
# derivadas: flotante. Derivada del sesgo
def derivadas_sesgo(datos):
    global g_filas
    restas = datos.map(lambda fila: (fila[2] - fila[1])) # (ŷ - y)
    suma   = restas.reduce(lambda x, y: x + y)
    dsesgo = 1/g_filas * suma
    return dsesgo
    
# datos: RDD de 1M de tuplas (X, y, ŷ)
# derivadas: np.array. Derivadas de los pesos (11, una por columna)
def derivadas_pesos(datos):
    derivadas = datos.map(lambda fila: (fila[2] - fila[1]) * fila[0]) # (ŷ - y) * X
    suma      = derivadas.reduce(lambda x, y: x + y)
    derivadas = suma / g_filas # np.array de 11 derivadas de los pesos
    return derivadas

# g_pesos: Array de (1Mx1)
# g_sesgo: flotante
# datos: RDD de 1M de regs. Cada reg: tupla (X, y) donde X es un array de (11x1)
# datos x sesgo: Array de (1Mx1)
# La multiplicación se puede hacer por filas y no se altera el resultado
# El resultado es que datos ahora es el mismo RDD pero las tuplas son (X, y, ŷ),
#  donde ŷ es la predicción de resultado
def sigmoide(datos):
    global g_pesos, g_sesgo
    entrada = datos.map(lambda fila: (fila[0], fila[1], np.matmul(fila[0], g_pesos) + g_sesgo))
    salida  = entrada.map(lambda fila: (fila[0], fila[1], 1/(1 + np.exp(-fila[2]))))
    return salida

def entrenar(datos, iteraciones, aprendizaje, tasareg):
    global g_filas, g_cols
    global g_pesos, g_dpesos
    global g_sesgo, g_dsesgo
    f = g_filas # 1.000.000
    c = g_cols  # 11
    g_pesos  = np.random.random([g_cols, ])
    g_dpesos = np.zeros([g_cols, ])
    for it in range(iteraciones): # (las siguientes dos líneas irían dentro del for)
        t1     = time.time()
        preds    = sigmoide(datos) # RDD 1M (X, y, ŷ)
        tpred  = time.time()
        g_dpesos = derivadas_pesos(preds)
        tderiv = time.time()
        g_dsesgo = derivadas_sesgo(preds) # Calcular la derivada del sesgo: 1/f*np.sum(pred_res-res)
        g_pesos  = g_pesos - (aprendizaje * g_dpesos) # w  = w - (aprendizaje * dw)
        g_sesgo  = g_sesgo - (aprendizaje * g_dsesgo) # b  = b - (aprendizaje * db)
        coste    = presio(preds, g_filas)
        t2 = time.time()
        print(str(it) + ": " + str(coste) + "[total: {} | sig: {} | derivs: {}]".format(t2-t1, tpred-t1, tderiv-tpred))
    # Calcular función de coste en cada iteración
    return derivs

# Pruebas

In [5]:
t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 4.129488468170166 segundos.


In [6]:
t1 = time.time()
datos = normalizar(rdd)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
# Haz la media de uno de los X. Tiene que ser ~0
# print(np.mean(datos.take(1)[0][0])) # 0.000000000000000010092936587501423
# La desviación estándar tiene que ser ~1
# print(np.std(datos.take(1)[0][0])) # 1.0

# 1.2345e-05 = 0.000012345

Tiempo transcurrido: 4.649162292480469e-05 segundos.


In [7]:
t1 = time.time()
derivadas = entrenar(datos, ITER, APRN, LMBD)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

0: 1e-05[total: 37.89878606796265 | sig: 1.3589859008789062e-05 | derivs: 20.81907629966736]
1: 1e-05[total: 34.50687766075134 | sig: 1.4781951904296875e-05 | derivs: 17.28230094909668]
2: 1e-05[total: 40.45198130607605 | sig: 1.6689300537109375e-05 | derivs: 23.982903957366943]
3: 1e-05[total: 37.306127071380615 | sig: 1.1920928955078125e-05 | derivs: 20.74366807937622]
4: 1e-05[total: 33.555511713027954 | sig: 2.3365020751953125e-05 | derivs: 17.182433605194092]
5: 1e-05[total: 33.61987924575806 | sig: 1.239776611328125e-05 | derivs: 16.886659145355225]
6: 1e-05[total: 33.304755210876465 | sig: 2.2172927856445312e-05 | derivs: 17.08894443511963]


KeyboardInterrupt: 

# derivs
print(type(derivadas))
print("1000000") # aux.count())
print(derivadas.take(1))

t1 = time.time()
red = derivadas.reduce(lambda x, y: x + y)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

t1 = time.time()
derivadas_final = red / g_filas
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
print(derivadas_final)